In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Lecture des données à partir du fichier Excel
df = pd.read_excel('surprise.xlsx')

# Afficher les catégories uniques de 'type_news'
print(df['type_news'].unique())

# Ajouter la catégorie manquante si nécessaire
categories = np.unique(np.append(df['type_news'].unique(), 'اخبار_وطنية'))

# Conversion de 'type_news' en encodage one-hot avec les catégories complètes
encoder = OneHotEncoder(categories=[categories])
type_news_encoded = encoder.fit_transform(df[['type_news']]).toarray()

# Utilisation de TfidfVectorizer pour convertir 'Titre de la page et classe de l'écran' en matrice TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500)
titre_tfidf = tfidf_vectorizer.fit_transform(df['Titre de la page et classe de l\'écran']).toarray()

# Combinaison des données traitées
X = np.concatenate((titre_tfidf, type_news_encoded), axis=1)
y_vues = df['Vues'].values
y_users = df['Utilisateurs'].values  # Assurez-vous que la colonne 'Utilisateurs' existe dans votre fichier Excel

# Combinaison des deux cibles
y = np.column_stack((y_vues, y_users))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


['اخبار_وطنية' 'المغرب-العالمي' 'regions' 'economie' 'politique'
 'art-et-culture' 'صحة']


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Construction du modèle de réseau de neurones à deux sorties
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='linear'))  # Deux sorties pour Vues et Utilisateurs

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1788897.6250 - mean_squared_error: 1788897.6250 - val_loss: 572677.0625 - val_mean_squared_error: 572677.0625
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2479533.7500 - mean_squared_error: 2479533.7500 - val_loss: 558649.8750 - val_mean_squared_error: 558649.8750
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9437013.0000 - mean_squared_error: 9437013.0000 - val_loss: 551926.9375 - val_mean_squared_error: 551926.9375
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3335570.7500 - mean_squared_error: 3335570.7500 - val_loss: 546422.2500 - val_mean_squared_error: 546422.2500
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2370405.7500 - mean_squared_error: 2370405.7500 - val_loss: 541664.9375 - val_mean_squared_error: 541664.9375
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6164300.5000 - mean_squared_error: 6164300.5000 - val_loss: 536163.0000 - val_mean_squared_error: 536

In [3]:
from sklearn.metrics import mean_squared_error, r2_score

# Faire des prédictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Évaluation du modèle pour les vues
mse_train_vues = mean_squared_error(y_train[:, 0], y_pred_train[:, 0])
r2_train_vues = r2_score(y_train[:, 0], y_pred_train[:, 0])
mse_test_vues = mean_squared_error(y_test[:, 0], y_pred_test[:, 0])
r2_test_vues = r2_score(y_test[:, 0], y_pred_test[:, 0])

print(f"Erreur quadratique moyenne d'entraînement pour les vues: {mse_train_vues}")
print(f"Score R^2 d'entraînement pour les vues: {r2_train_vues}")
print(f"Erreur quadratique moyenne de test pour les vues: {mse_test_vues}")
print(f"Score R^2 de test pour les vues: {r2_test_vues}")

# Évaluation du modèle pour les utilisateurs
mse_train_users = mean_squared_error(y_train[:, 1], y_pred_train[:, 1])
r2_train_users = r2_score(y_train[:, 1], y_pred_train[:, 1])
mse_test_users = mean_squared_error(y_test[:, 1], y_pred_test[:, 1])
r2_test_users = r2_score(y_test[:, 1], y_pred_test[:, 1])

print(f"Erreur quadratique moyenne d'entraînement pour les utilisateurs: {mse_train_users}")
print(f"Score R^2 d'entraînement pour les utilisateurs: {r2_train_users}")
print(f"Erreur quadratique moyenne de test pour les utilisateurs: {mse_test_users}")
print(f"Score R^2 de test pour les utilisateurs: {r2_test_users}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Erreur quadratique moyenne d'entraînement pour les vues: 5146172.68372945
Score R^2 d'entraînement pour les vues: 0.42011486656500896
Erreur quadratique moyenne de test pour les vues: 635821.8786777294
Score R^2 de test pour les vues: -1.4965125687823453
Erreur quadratique moyenne d'entraînement pour les utilisateurs: 603794.053016146
Score R^2 d'entraînement pour les utilisateurs: 0.4988372424816183
Erreur quadratique moyenne de test pour les utilisateurs: 223970.08186199056
Score R^2 de test pour les utilisateurs: -0.7751495146755025


In [4]:
import pickle

# Sauvegarder le modèle
model.save('my_model.h5')

# Sauvegarder l'encoder
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

# Sauvegarder le tfidf_vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
